In [1]:
import re
import os
import json
import numpy as np
from tqdm import tqdm
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from utils.encode import GenericPLMEncoder

# Create the elastic instance
elastic = Elasticsearch(
    "http://localhost:9200",
    request_timeout=1000000
)

# Successful response!
elastic.info()

ObjectApiResponse({'name': 'ubuntu', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'G7JzeSt3Q5OoTS6zBdAaWQ', 'version': {'number': '8.4.1', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '2bd229c8e56650b42e40992322a76e7914258f0c', 'build_date': '2022-08-26T12:11:43.232597118Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [ ]:
emb = np.random.rand(768).astype(np.float32)

resp = elastic.search(
    index="case",
    size=2,
    from_=0,
    _source=True,
    fields=["case_name", "content", {"field": "publish_date", "format": "year_month_day"}, "court_name", "case_type", "id", "legal_base", "pub_prosecution_org"],
    query={
        # "match": {
        #     "content": "2016年2月26日，原告的员工郑洲招用被告到原告工地做工，并签订劳动合同。被告工作的时间是2016年2月26日至2016年9月21日。2016年9月26日，被告与原告决算，拿工钱离开。被告在工作期间由郑洲管理。",
        # },
        # "match_phrase": {
        #     "case_name": "彭绍波与陈忠远民间借贷纠纷一审民事判决书",
        # },
        "combined_fields": {
            "query": "借贷",
            "fields": ["case_name", "content"],
        },
        # "multi_match" : {
        #     "query": "打了一个借条",
        #     "type": "phrase",
        #     "fields": ["case_name", "content"]
        # },
        # "script_score": {
        #     "query": {
        #         "match": {
        #             "content": "2016年2月26日，原告的员工郑洲招用被告到原告工地做工，并签订劳动合同。被告工作的时间是2016年2月26日至2016年9月21日。2016年9月26日，被告与原告决算，拿工钱离开。被告在工作期间由郑洲管理。"
        #         }
        #     },
        #     "script": {
        #         # "source": "_score"
        #         "source": "doc['vector'].size() == 0 ? 0 : dotProduct(params.queryVector, 'vector') + 1",
        #         "params": {
        #             "queryVector": a
        #         }
        #     },
        # },
    },
    # knn={
    #     "field": "vector",
    #     "query_vector": (emb / np.linalg.norm(emb)).tolist(),
    #     "k": 10,
    #     "num_candidates": 10,
    #     # "boost": 1
    # },
    # aggs={
    #     "agg-court": {
    #         "terms": {
    #             "field": "legal_base"
    #         }
    #     }
    # },
    # highlight={
    #     "fields": {
    #         "content": {
    #             "pre_tags" : ["<strong>"],
    #             "post_tags": ["</strong>"],
    #             "number_of_fragments": 1,
    #         }
    #     }
    # },
    # post_filter={
    #     "bool": {
    #         "filter": [
    #             {
    #                 "terms": {"legal_base": ["《中华人民共和国合同法》第二百零六条"]}
    #             }
    #         ]
    #     }
    # }
)

In [ ]:
a = elastic.get(index="test", id="hpCUWoQB1-2U-nhAcL7L")

In [3]:
elastic.indices.create(
    index="case",
    settings={
        'analysis': {
            'analyzer': {
                # we must set the default analyzer
                "default": {
                    "type": "smartcn"
                }
            }
        },
        "index.mapping.ignore_malformed": True
    },
    mappings={
        "properties": {
            # field name
            "collection": {
                "type": "keyword",
            },
            "id": {
                "type": "unsigned_long",
            },
            "court_name": {
                "type": "keyword",
            },
            "court_province": {
                "type": "keyword",
            },
            "court_city": {
                "type": "keyword",
            },
            "court_region": {
                "type": "keyword",
            },
            "court_district": {
                "type": "keyword"
            },
            "pub_prosecution_org": {
                "type": "keyword"
            },
            "case_type": {
                "type": "keyword",
            },
            "cause": {
                "type": "keyword",
            },
            "trial_round": {
                "type": "keyword"
            },
            "content": {
                "type": "text"
            },
            "appellor": {
                "type": "keyword"
            },
            "legal_base": {
                "type": "keyword"
            },
            "lawyer": {
                "type": "keyword"
            },
            "law_firm": {
                "type": "keyword"
            },
            "admin_behavior_type": {
                "type": "keyword"
            },
            "admin_manage_scope": {
                "type": "keyword"
            },
            "effect_level": {
                "type": "keyword"
            },
            "vector": {
                "type": "dense_vector",
                "dims": 768,
                # enable hnsw
                "index": True,
                # inner product only allows unit-length vector
                "similarity": "dot_product"  
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'case'})

In [ ]:
elastic.index(index="test", document={"content": "这是为什么捏？我也不知道捏"}, id="shit")

In [ ]:
elastic.get(index="test", id="n--UYIQBP9S5ZJCvVnHk")

In [2]:
# delete index
elastic.indices.delete(index="case")

ObjectApiResponse({'acknowledged': True})

In [4]:
elastic.count(
    index="case",
    query={
        "match_all": {
        }
    }
)

ObjectApiResponse({'count': 0, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [ ]:
elastic.indices.get_mapping(index="wenshu")

In [ ]:
elastic.indices.get_alias(index="*")

In [ ]:
elastic.indices.analyze(text="知识产权高校", index="case")

In [ ]:
elastic.delete_by_query(
    index="wenshu", 
    query={
        "match_all": {}
    }
)

In [ ]:
import re
import json
from transformers import AutoModel, AutoTokenizer
from utils.encode import GenericPLMEncoder

plm = AutoModel.from_pretrained("/home/peitian_zhang/Code/Case-Retriever/src/data/model/DPR")
tokenizer = AutoTokenizer.from_pretrained("/home/peitian_zhang/Code/Case-Retriever/src/data/model/DPR")
model = GenericPLMEncoder(plm, tokenizer)

a = model.encode_single_query("""

2016年2月26日，原告的员工郑洲招用被告到原告工地做工，并签订劳动合同。被告工作的时间是2016年2月26日至2016年9月21日。2016年9月26日，被告与原告决算，拿工钱离开。被告在工作期间由郑洲管理。
庭审中，原告举示借条，该借条主要载明：今收到原告支付中交·锦悦一期二组团三标段工程5#楼（二十层-三十层）木工班组借款，金额5000元。被告在领款人签字处签名，时间为2016年8月26日。借条上有原告公章。被告质证称借条上确实是被告签名，但是出具借条时原告公司没有加盖公章，原告公司的公章是工程结束做决算时，原告在工资中抵扣了该笔款项后加盖的。
庭审中，本院向原、被告释明，本案不属于民间借贷或借款合同法律关系，原告与被告存在劳动关系，本案应属劳动争议。原告当庭表示不认可本院的释明，坚持本案属于民间借贷纠纷，仍坚持以民间借贷法律关系提出请求。
上述事实，有借条、重庆市江北区人民法院庭审笔录2份以及当事人的陈述等证据在卷佐证，并经庭审质证。

""".replace("\n", ""))

In [ ]:
# elastic.indices.delete(index="test")

elastic.indices.create(
    index="test",
    mappings={
        "properties": {
            # field name
            "x": {
                "type": "text",
            }
        }
    }
)

In [ ]:
for i, x in enumerate(f):
    doc = {
        "x": [f"{i}", f"{i+1}", f"{i+2}"],
        "content": f"This is the {i}-th document"
    }
    elastic.index(index="test", document=doc)
    if i > 10:
        break

In [ ]:
resp = elastic.search(
    index="test",
    size=10,
    _source=False,
    fields=["x", "content"],
    query={
        # "bool": {
        #     "must": [
        #         # {"term": {"content": "document"}},
        #         {"term": {"content": "this"}},            
        #         {"term": {"content": "0"}},            
        #     ],
        # },
        "match": {
            "content": {
                "query": "this is the 0-th",
                "operator": "AND"
            }
        }
    },
)

In [ ]:
resp["hits"]